## Скачивание данных для тренировки модели

In [ ]:
# подключение к хранилищу Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# распаковка и скачивание архива с тренировочными данными в локальное хранилище

!unzip gdrive/My\ Drive/data/Diploma\ ML-mid/train.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/uncertain/1832.jpg  
  inflating: train/uncertain/1833.jpg  
  inflating: train/uncertain/1834.jpg  
  inflating: train/uncertain/1835.jpg  
  inflating: train/uncertain/1836.jpg  
  inflating: train/uncertain/1837.jpg  
  inflating: train/uncertain/1838.jpg  
  inflating: train/uncertain/1839.jpg  
  inflating: train/uncertain/184.jpg  
  inflating: train/uncertain/1840.jpg  
  inflating: train/uncertain/1841.jpg  
  inflating: train/uncertain/1842.jpg  
  inflating: train/uncertain/1843.jpg  
  inflating: train/uncertain/1844.jpg  
  inflating: train/uncertain/1845.jpg  
  inflating: train/uncertain/1846.jpg  
  inflating: train/uncertain/1847.jpg  
  inflating: train/uncertain/1848.jpg  
  inflating: train/uncertain/1849.jpg  
  inflating: train/uncertain/185.jpg  
  inflating: train/uncertain/1850.jpg  
  inflating: train/uncertain/1851.jpg  
  inflating: train/uncertain/1852.jpg  
  inflating: t

## Подготовка тренировочной и валидационной выборок

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 100
IMG_SHAPE  = 128 

dir = Path("train") # путь до папки с изображениями

In [ ]:
# генератор данных
image_gen = ImageDataGenerator(
                              validation_split=0.2 # размер валидационной выборки
                              )   

# тренировочный генератор
train_data_gen = image_gen.flow_from_directory(batch_size=BATCH_SIZE, # размер батча
                                               directory=dir, # путь до папки с изображениями
                                               shuffle=True, # перемешивание данных 
                                               target_size=(IMG_SHAPE,IMG_SHAPE), # размер к которому будут приведены все изображения
                                               class_mode="categorical", # так как классов 9
                                               subset = "training") # тип выборки (случай, когда и тренировочные, и валидационные данные находятся в 1 директории)
# валидационный генератор
val_data_gen = image_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                             directory=dir,
                                             shuffle=False,
                                             target_size=(IMG_SHAPE,IMG_SHAPE),
                                             class_mode='categorical',
                                             subset = "validation")

Found 40039 images belonging to 9 classes.
Found 10007 images belonging to 9 classes.


## Создание, обучение и сохранение модели

In [ ]:
from keras.optimizers import Adam,SGD,RMSprop
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from keras.optimizers import Adam,SGD,RMSprop

NUM_CLASSES = 9

model = Sequential()

# 1-ый CNN блок
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (IMG_SHAPE,IMG_SHAPE,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

# 2-ой CNN блок
model.add(Conv2D(128,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

# 3-ий CNN блок
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

# 4-ый CNN блок
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

# 1-ый полносвязный блок 
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# 2-ой полносвязный блок
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# выходной слой
model.add(Dense(NUM_CLASSES, activation='softmax'))

opt = Adam(lr = 0.0001)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 64)      256       
_________________________________________________________________
activation (Activation)      (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 128)       5

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("./model.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# остановка обучения когда метрика перестала улучшаться
early_stopping = EarlyStopping(monitor='val_loss', # метрика для отслеживания
                               min_delta=0, # порог изменение "монитора" для оценки эпохи
                               patience=3, # сколько эпох может отсутсвовать улучшение "монитора"
                               verbose=1, # что печатается в блоке вывода
                               restore_best_weights=True # восстановить веса модели с момента эпохи с лучшим значением "монитора"
                               )

# уменьшение шага обучения когда метрика перестала улучшаться
reduce_learningrate = ReduceLROnPlateau(monitor='val_loss', # метрика для отслеживания
                                        factor=0.2, # как изменится шаг обучения (новый шаг = старый шаг * factor)
                                        patience=3, # сколько эпох может отсутсвовать улучшение "монитора"
                                        verbose=1, # что печатается в блоке вывода
                                        min_delta=0.0001 # порог изменение "монитора" для оценки эпохи
                                        )

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

EPOCHS = 30

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# обучение модели
history = model.fit_generator(generator=train_data_gen,
                              steps_per_epoch=train_data_gen.n//train_data_gen.batch_size, # общее количество шагов для 1 эпохи
                              epochs=EPOCHS,
                              validation_data = val_data_gen,
                              validation_steps = val_data_gen.n//val_data_gen.batch_size, # общее количество шагов для 1 эпохи (на валидации)
                              callbacks=callbacks_list)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
400/400 [==============================] - 469s 1s/step - loss: 2.3335 - accuracy: 0.1563 - val_loss: 1.9021 - val_accuracy: 0.3029
Epoch 2/30
400/400 [==============================] - 407s 1s/step - loss: 1.8737 - accuracy: 0.3169 - val_loss: 1.7137 - val_accuracy: 0.3705
Epoch 3/30
400/400 [==============================] - 392s 981ms/step - loss: 1.6382 - accuracy: 0.4055 - val_loss: 1.6208 - val_accuracy: 0.4009
Epoch 4/30
400/400 [==============================] - 396s 992ms/step - loss: 1.5356 - accuracy: 0.4385 - val_loss: 1.5613 - val_accuracy: 0.4373
Epoch 5/30
400/400 [==============================] - 403s 1s/step - loss: 1.4279 - accuracy: 0.4809 - val_loss: 1.5466 - val_accuracy: 0.4362
Epoch 6/30
400/400 [==============================] - 403s 1s/step - loss: 1.3335 - accuracy: 0.5157 - val_loss: 1.5706 - val_accuracy: 0.4359
Epoch 7/30
400/400 [==============================] - 406s 1s/step - loss: 1.2524 - accuracy: 0.5455 - val_loss: 1.5452 - val_accuracy: 

In [ ]:
# сохранение модели

model.save("/content/gdrive/MyDrive/models/Diploma ML-mid (default)/Custom")

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/models/Diploma ML-mid (default)/Custom/assets


## Проверка времени инференса модели

In [ ]:
import numpy as np
import cv2
from imageio import imread

# подготовка изображения для классификации 

data = np.empty((1, 128, 128, 3))

image = cv2.imread("/content/gdrive/MyDrive/data/Diploma ML-mid/test_kaggle/4914.jpg")
image = cv2.resize(image, (128, 128))
data[0] = image
data = preprocess_input(data)

In [ ]:
# среднее время работы
%%timeit -n 10 -r 10

predictions = model.predict(data)

10 loops, best of 10: 48.7 ms per loop


In [ ]:
# предсказание и определение индекса класса

predictions = model.predict(data)
np.argmax(predictions)

4